# Project - Aegea

## Imports and packages

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import folium

In [ ]:
df = pd.read_csv("./data_source/amostra_menor.csv", sep=";")

In [ ]:
df_c = df.copy()

## Functions

In [ ]:
def radians(series):
    return np.radians(series)

## Data cleaning

In [ ]:
df_c.info()

In [ ]:
df_c.head()

In [ ]:
df_c['LATITUDE_RAD'] = df_c['LATITUDE'].apply(radians)
df_c['LONGITUDE_RAD'] = df_c['LONGITUDE'].apply(radians)

In [ ]:
coords = df_c[['LATITUDE_RAD', 'LONGITUDE_RAD']].to_numpy()

In [ ]:
# Utilizando DBSCAN para clusterização baseada em distância haversine
# Ajuste os parâmetros eps e min_samples conforme necessário
kms_per_radian = 6371.0088  # Raio médio da Terra em km
epsilon = 1.5 / kms_per_radian  # Distância em km convertida para radianos

In [ ]:
epsilon

In [ ]:
db = DBSCAN(eps=0.00000544152065839246, min_samples=5, algorithm='ball_tree', metric='haversine')
clusters = db.fit_predict(coords)

In [ ]:
df_c['cluster'] = clusters

In [ ]:
df_c['cluster'].nunique()

In [ ]:
df_c['cluster'].unique()

## Plot map

In [ ]:
# Cria um mapa com um ponto central. Os valores centrais aqui são as médias das latitudes e longitudes
mapa = folium.Map(location=[df_c['LATITUDE'].mean(), df_c['LONGITUDE'].mean()], zoom_start=12)

# Paleta de cores para os clusters
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Adicionando marcadores ao mapa
for _, row in df_c.iterrows():
    folium.CircleMarker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        radius=5,
        color=colors[row['cluster'] % len(colors)],
        fill=True,
        fill_color=colors[row['cluster'] % len(colors)],
        fill_opacity=0.7,
        popup=f"Cluster: {row['cluster']}"
    ).add_to(mapa)

# Salvar o mapa em um arquivo HTML
mapa.save('./maps/clustering_map.html')